In [65]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [13]:
sample = pd.read_csv("dataset/sit/downSit/downSit1.csv", sep = ";")
sample.head()

,Date,Timestamp,DeviceOrientation,AccelerationX,AccelerationY,AccelerationZ,Label
0,2020-07-14 14:55:48,1.594764e+09,portrait,-0.425507,0.624695,-0.630051,NaN
1,2020-07-14 14:55:48,1.594764e+09,portrait,-0.424881,0.626556,-0.579483,NaN
2,2020-07-14 14:55:48,1.594764e+09,portrait,-0.429794,0.634567,-0.610840,NaN
3,2020-07-14 14:55:48,1.594764e+09,portrait,-0.430939,0.653580,-0.642944,NaN
4,2020-07-14 14:55:48,1.594764e+09,portrait,-0.429077,0.630432,-0.622650,NaN


In [25]:
sample["DeviceOrientation"].unique()

numpy.ndarray

In [35]:
x = list(sample["DeviceOrientation"].unique())

In [36]:
DeviceOrientations = set()
DeviceOrientations.update(x)
DeviceOrientations

{'faceUp', 'portrait', 'portraitUpsideDown'}

In [44]:
sample["DeviceOrientation"].unique()

array(['portrait', 'portraitUpsideDown', 'faceUp'], dtype=object)

In [47]:
sample.values

array([['2020-07-14 14:55:48', 1594763748.773793, 'portrait', ...,
        0.62469482421875, -0.6300506591796875, nan],
       ['2020-07-14 14:55:48', 1594763748.778411, 'portrait', ...,
        0.626556396484375, -0.5794830322265625, nan],
       ['2020-07-14 14:55:48', 1594763748.782727, 'portrait', ...,
        0.6345672607421875, -0.61083984375, nan],
       ...,
       ['2020-07-14 14:55:48', 1594763789.640328, 'faceUp', ...,
        -0.28643798828125, -0.9506072998046876, nan],
       ['2020-07-14 14:55:48', 1594763789.679858, 'faceUp', ...,
        -0.261444091796875, -0.9839630126953124, nan],
       ['2020-07-14 14:55:48', 1594763789.719836, 'faceUp', ...,
        -0.2554779052734375, -1.0263824462890625, nan]], dtype=object)

In [55]:
X = sample.drop(['Date', 'Timestamp', 'Label'], axis=1)
X = X.replace(encoder)
X.head()

,DeviceOrientation,AccelerationX,AccelerationY,AccelerationZ
0,4,-0.425507,0.624695,-0.630051
1,4,-0.424881,0.626556,-0.579483
2,4,-0.429794,0.634567,-0.610840
3,4,-0.430939,0.653580,-0.642944
4,4,-0.429077,0.630432,-0.622650


In [56]:
X["Class"] = 0

In [57]:
X.head()

,DeviceOrientation,AccelerationX,AccelerationY,AccelerationZ,Class
0,4,-0.425507,0.624695,-0.630051,0
1,4,-0.424881,0.626556,-0.579483,0
2,4,-0.429794,0.634567,-0.610840,0
3,4,-0.430939,0.653580,-0.642944,0
4,4,-0.429077,0.630432,-0.622650,0


In [14]:
cols = list(sample.columns)

In [12]:
classes = os.listdir("dataset")
classes

['fall', 'sit']

In [20]:
# class distribution

cls_count = {classes[0]: 0, classes[1]: 0}
cls_count

for cls in classes:
    sub_classes = os.listdir(f"dataset/{cls}")
    for sub_cls in sub_classes:
        files = os.listdir(f"dataset/{cls}/{sub_cls}")
        cls_count[cls] += len(files)

cls_count

{'fall': 108, 'sit': 95}

In [22]:
# checking number of attributes
for cls in classes:
    sub_classes = os.listdir(f"dataset/{cls}")
    for sub_cls in sub_classes:
        paths = os.listdir(f"dataset/{cls}/{sub_cls}")
        for path in paths:
            df = pd.read_csv(f"dataset/{cls}/{sub_cls}/{path}", sep = ";")
            if not cols == list(df.columns):
                print("Detected")

In [43]:
# checking number of DeviceOrientation types
DeviceOrientations = set()

for cls in classes:
    sub_classes = os.listdir(f"dataset/{cls}")
    for sub_cls in sub_classes:
        paths = os.listdir(f"dataset/{cls}/{sub_cls}")
        for path in paths:
            df = pd.read_csv(f"dataset/{cls}/{sub_cls}/{path}", sep = ";")
            DeviceOrientations.update(list(df["DeviceOrientation"].unique()))
  
DeviceOrientations = sorted(list(DeviceOrientations))
DeviceOrientations

['faceDown',
 'faceUp',
 'landscapeLeft',
 'landscapeRight',
 'portrait',
 'portraitUpsideDown']

In [54]:
encoder = {"DeviceOrientation": {}}

for i, orientation in enumerate(DeviceOrientations):
    encoder["DeviceOrientation"][orientation] = i

encoder

{'DeviceOrientation': {'faceDown': 0,
  'faceUp': 1,
  'landscapeLeft': 2,
  'landscapeRight': 3,
  'portrait': 4,
  'portraitUpsideDown': 5}}

In [69]:
# appending data frames and lebel encoding

X = []
y = []

for i, cls in enumerate(classes):
    sub_classes = os.listdir(f"dataset/{cls}")
    for sub_cls in sub_classes:
        paths = os.listdir(f"dataset/{cls}/{sub_cls}")
        for path in paths:
            df = pd.read_csv(f"dataset/{cls}/{sub_cls}/{path}", sep = ";")
            df = df.drop(['Date', 'Timestamp', 'Label'], axis=1)
            df = df.replace(encoder)
            X.append(df)
            y += [i] * len(df)

X = pd.concat(X)

In [70]:
X = X.values
len(X)

202599

In [71]:
len(y)

202599

In [72]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [73]:
SVM = SVC()
SVM.fit(X_train,y_train)

SVC()

In [74]:
y_pred = SVM.predict(X_test)
 
# compute accuracy of the model
SVM.score(X_test, y_test)

0.6892398815399803

In [75]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(y_test,y_pred)
cm

array([[7352, 3644],
       [2652, 6612]], dtype=int64)